In [1]:
!pip install --upgrade pip
!pip install --upgrade tflite-support

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 15.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import h5py

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model_path = 'drive/MyDrive/CapstoneML/model_ska.h5'

In [5]:
import json

def inspect_model_config(model_path):
    # Load the model file
    with h5py.File(model_path, 'r') as h5file:
        # Extract the model configuration
        model_config = json.loads(h5file.attrs['model_config'])  # Directly loading without decode
        print(json.dumps(model_config, indent=4))

In [6]:
inspect_model_config(model_path)

{
    "class_name": "Sequential",
    "config": {
        "name": "sequential",
        "trainable": true,
        "dtype": "float32",
        "layers": [
            {
                "class_name": "InputLayer",
                "config": {
                    "batch_shape": [
                        null,
                        224,
                        224,
                        3
                    ],
                    "dtype": "float32",
                    "sparse": false,
                    "name": "input_layer"
                }
            },
            {
                "class_name": "Conv2D",
                "config": {
                    "name": "conv2d",
                    "trainable": true,
                    "dtype": "float32",
                    "filters": 16,
                    "kernel_size": [
                        3,
                        3
                    ],
                    "strides": [
                        1,
                        1


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3), name='input_layer'),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', name='conv2d', padding='valid'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d'),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_1', padding='valid'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_1'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2', padding='valid'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_2'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', name='conv2d_3', padding='valid'),
    tf.keras.layers.MaxPooling2D((2, 2), name='max_pooling2d_3'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', name='dense'),
    tf.keras.layers.Dense(18, activation='softmax', name='dense_1')
])

In [8]:
model.load_weights('drive/MyDrive/CapstoneML/model_ska.h5')

In [9]:
saved_model_dir = 'saved_model_directory'
model.save(saved_model_dir, save_format='tf')

print(f"Model has been saved in TensorFlow SavedModel format at: {saved_model_dir}")

Model has been saved in TensorFlow SavedModel format at: saved_model_directory


In [10]:
tflife_converter = tf.lite.TFLiteConverter.from_saved_model('/content/saved_model_directory/')
tflife_model = tflife_converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tflife_model)

In [12]:
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

ImageClassifierWriter = image_classifier.MetadataWriter
_MODEL_PATH = "/content/model.tflite"
_NORMALIZATION_MEAN = 127.5
_NORMALIZATION_STD = 127.5

# Create the metadata writer and populate it without a label file.
writer = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), input_norm_mean=[_NORMALIZATION_MEAN],
    input_norm_std=[_NORMALIZATION_STD], label_file_paths=[])

writer_utils.save_file(writer.populate(), "model_with_metadata.tflite")